In [1]:
!pip install hopsworks

In [2]:
import datetime
import requests
import pandas as pd
import hopsworks
import datetime
from pathlib import Path
import json
import re
import os
import warnings
warnings.filterwarnings("ignore")

In [21]:
with open('hopsworks-api-key.txt', 'r') as file:
    os.environ["HOPSWORKS_API_KEY"] = file.read().rstrip()

project = hopsworks.login()

Connection closed.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1171606


In [33]:
df = pd.read_csv('Airline_Delay_Cause.csv', skipinitialspace=True)
df

,year,month,carrier,carrier_name,airport,airport_name,arr_flights,arr_del15,carrier_ct,weather_ct,...,security_ct,late_aircraft_ct,arr_cancelled,arr_diverted,arr_delay,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay
0,2024,9,9E,Endeavor Air Inc.,ATL,"Atlanta, GA: Hartsfield-Jackson Atlanta Intern...",2204.0,280.0,61.70,17.36,...,0.00,112.57,40.0,4.0,24383.0,7840.0,2297.0,3690.0,0.0,10556.0
1,2024,9,AA,American Airlines Network,ATL,"Atlanta, GA: Hartsfield-Jackson Atlanta Intern...",709.0,159.0,63.21,3.02,...,0.07,57.46,4.0,0.0,11435.0,5343.0,238.0,1327.0,6.0,4521.0
2,2024,9,AS,Alaska Airlines Network,ATL,"Atlanta, GA: Hartsfield-Jackson Atlanta Intern...",138.0,37.0,10.67,1.63,...,0.00,6.23,2.0,1.0,1859.0,606.0,135.0,907.0,0.0,211.0
3,2024,9,B6,JetBlue Airways,ATL,"Atlanta, GA: Hartsfield-Jackson Atlanta Intern...",276.0,57.0,26.27,0.98,...,0.00,10.91,0.0,1.0,3588.0,1981.0,40.0,657.0,0.0,910.0
4,2024,9,DL,Delta Air Lines Network,ATL,"Atlanta, GA: Hartsfield-Jackson Atlanta Intern...",19151.0,2429.0,709.98,122.72,...,0.68,904.72,101.0,30.0,163868.0,68972.0,12036.0,26112.0,32.0,56716.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1600,2014,7,MQ,Envoy Air,ATL,"Atlanta, GA: Hartsfield-Jackson Atlanta Intern...",81.0,21.0,3.28,0.20,...,0.00,6.13,5.0,1.0,843.0,78.0,8.0,535.0,0.0,222.0
1601,2014,7,OO,SkyWest Airlines Inc.,ATL,"Atlanta, GA: Hartsfield-Jackson Atlanta Intern...",198.0,48.0,16.68,1.67,...,0.00,11.13,4.0,3.0,2173.0,821.0,230.0,604.0,0.0,518.0
1602,2014,7,UA,United Air Lines Inc.,ATL,"Atlanta, GA: Hartsfield-Jackson Atlanta Intern...",61.0,37.0,15.79,0.28,...,0.00,9.41,0.0,1.0,1665.0,652.0,13.0,386.0,0.0,614.0
1603,2014,7,US,US Airways Inc.,ATL,"Atlanta, GA: Hartsfield-Jackson Atlanta Intern...",474.0,111.0,55.69,7.03,...,1.20,18.17,3.0,0.0,5871.0,3043.0,757.0,1193.0,32.0,846.0


In [34]:
import pandas as pd



# Combine 'year' and 'month' columns into a 'year_month' column in 'YYYY-MM' format
df['year'] = df['year'].astype(int)
df['month'] = df['month'].astype(int)

df


,year,month,carrier,carrier_name,airport,airport_name,arr_flights,arr_del15,carrier_ct,weather_ct,...,security_ct,late_aircraft_ct,arr_cancelled,arr_diverted,arr_delay,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay
0,2024,9,9E,Endeavor Air Inc.,ATL,"Atlanta, GA: Hartsfield-Jackson Atlanta Intern...",2204.0,280.0,61.70,17.36,...,0.00,112.57,40.0,4.0,24383.0,7840.0,2297.0,3690.0,0.0,10556.0
1,2024,9,AA,American Airlines Network,ATL,"Atlanta, GA: Hartsfield-Jackson Atlanta Intern...",709.0,159.0,63.21,3.02,...,0.07,57.46,4.0,0.0,11435.0,5343.0,238.0,1327.0,6.0,4521.0
2,2024,9,AS,Alaska Airlines Network,ATL,"Atlanta, GA: Hartsfield-Jackson Atlanta Intern...",138.0,37.0,10.67,1.63,...,0.00,6.23,2.0,1.0,1859.0,606.0,135.0,907.0,0.0,211.0
3,2024,9,B6,JetBlue Airways,ATL,"Atlanta, GA: Hartsfield-Jackson Atlanta Intern...",276.0,57.0,26.27,0.98,...,0.00,10.91,0.0,1.0,3588.0,1981.0,40.0,657.0,0.0,910.0
4,2024,9,DL,Delta Air Lines Network,ATL,"Atlanta, GA: Hartsfield-Jackson Atlanta Intern...",19151.0,2429.0,709.98,122.72,...,0.68,904.72,101.0,30.0,163868.0,68972.0,12036.0,26112.0,32.0,56716.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1600,2014,7,MQ,Envoy Air,ATL,"Atlanta, GA: Hartsfield-Jackson Atlanta Intern...",81.0,21.0,3.28,0.20,...,0.00,6.13,5.0,1.0,843.0,78.0,8.0,535.0,0.0,222.0
1601,2014,7,OO,SkyWest Airlines Inc.,ATL,"Atlanta, GA: Hartsfield-Jackson Atlanta Intern...",198.0,48.0,16.68,1.67,...,0.00,11.13,4.0,3.0,2173.0,821.0,230.0,604.0,0.0,518.0
1602,2014,7,UA,United Air Lines Inc.,ATL,"Atlanta, GA: Hartsfield-Jackson Atlanta Intern...",61.0,37.0,15.79,0.28,...,0.00,9.41,0.0,1.0,1665.0,652.0,13.0,386.0,0.0,614.0
1603,2014,7,US,US Airways Inc.,ATL,"Atlanta, GA: Hartsfield-Jackson Atlanta Intern...",474.0,111.0,55.69,7.03,...,1.20,18.17,3.0,0.0,5871.0,3043.0,757.0,1193.0,32.0,846.0


In [35]:

df = df.drop(columns=['airport','airport_name'])
df

,year,month,carrier,carrier_name,arr_flights,arr_del15,carrier_ct,weather_ct,nas_ct,security_ct,late_aircraft_ct,arr_cancelled,arr_diverted,arr_delay,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay
0,2024,9,9E,Endeavor Air Inc.,2204.0,280.0,61.70,17.36,88.36,0.00,112.57,40.0,4.0,24383.0,7840.0,2297.0,3690.0,0.0,10556.0
1,2024,9,AA,American Airlines Network,709.0,159.0,63.21,3.02,35.25,0.07,57.46,4.0,0.0,11435.0,5343.0,238.0,1327.0,6.0,4521.0
2,2024,9,AS,Alaska Airlines Network,138.0,37.0,10.67,1.63,18.47,0.00,6.23,2.0,1.0,1859.0,606.0,135.0,907.0,0.0,211.0
3,2024,9,B6,JetBlue Airways,276.0,57.0,26.27,0.98,18.84,0.00,10.91,0.0,1.0,3588.0,1981.0,40.0,657.0,0.0,910.0
4,2024,9,DL,Delta Air Lines Network,19151.0,2429.0,709.98,122.72,690.90,0.68,904.72,101.0,30.0,163868.0,68972.0,12036.0,26112.0,32.0,56716.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1600,2014,7,MQ,Envoy Air,81.0,21.0,3.28,0.20,11.39,0.00,6.13,5.0,1.0,843.0,78.0,8.0,535.0,0.0,222.0
1601,2014,7,OO,SkyWest Airlines Inc.,198.0,48.0,16.68,1.67,18.52,0.00,11.13,4.0,3.0,2173.0,821.0,230.0,604.0,0.0,518.0
1602,2014,7,UA,United Air Lines Inc.,61.0,37.0,15.79,0.28,11.51,0.00,9.41,0.0,1.0,1665.0,652.0,13.0,386.0,0.0,614.0
1603,2014,7,US,US Airways Inc.,474.0,111.0,55.69,7.03,28.91,1.20,18.17,3.0,0.0,5871.0,3043.0,757.0,1193.0,32.0,846.0


In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1605 entries, 0 to 1604
Data columns (total 19 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   year                 1605 non-null   int64  
 1   month                1605 non-null   int64  
 2   carrier              1605 non-null   object 
 3   carrier_name         1605 non-null   object 
 4   arr_flights          1605 non-null   float64
 5   arr_del15            1604 non-null   float64
 6   carrier_ct           1605 non-null   float64
 7   weather_ct           1605 non-null   float64
 8   nas_ct               1605 non-null   float64
 9   security_ct          1605 non-null   float64
 10  late_aircraft_ct     1605 non-null   float64
 11  arr_cancelled        1605 non-null   float64
 12  arr_diverted         1605 non-null   float64
 13  arr_delay            1605 non-null   float64
 14  carrier_delay        1605 non-null   float64
 15  weather_delay        1605 non-null   f

In [37]:
df.dropna(inplace=True)
df

,year,month,carrier,carrier_name,arr_flights,arr_del15,carrier_ct,weather_ct,nas_ct,security_ct,late_aircraft_ct,arr_cancelled,arr_diverted,arr_delay,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay
0,2024,9,9E,Endeavor Air Inc.,2204.0,280.0,61.70,17.36,88.36,0.00,112.57,40.0,4.0,24383.0,7840.0,2297.0,3690.0,0.0,10556.0
1,2024,9,AA,American Airlines Network,709.0,159.0,63.21,3.02,35.25,0.07,57.46,4.0,0.0,11435.0,5343.0,238.0,1327.0,6.0,4521.0
2,2024,9,AS,Alaska Airlines Network,138.0,37.0,10.67,1.63,18.47,0.00,6.23,2.0,1.0,1859.0,606.0,135.0,907.0,0.0,211.0
3,2024,9,B6,JetBlue Airways,276.0,57.0,26.27,0.98,18.84,0.00,10.91,0.0,1.0,3588.0,1981.0,40.0,657.0,0.0,910.0
4,2024,9,DL,Delta Air Lines Network,19151.0,2429.0,709.98,122.72,690.90,0.68,904.72,101.0,30.0,163868.0,68972.0,12036.0,26112.0,32.0,56716.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1600,2014,7,MQ,Envoy Air,81.0,21.0,3.28,0.20,11.39,0.00,6.13,5.0,1.0,843.0,78.0,8.0,535.0,0.0,222.0
1601,2014,7,OO,SkyWest Airlines Inc.,198.0,48.0,16.68,1.67,18.52,0.00,11.13,4.0,3.0,2173.0,821.0,230.0,604.0,0.0,518.0
1602,2014,7,UA,United Air Lines Inc.,61.0,37.0,15.79,0.28,11.51,0.00,9.41,0.0,1.0,1665.0,652.0,13.0,386.0,0.0,614.0
1603,2014,7,US,US Airways Inc.,474.0,111.0,55.69,7.03,28.91,1.20,18.17,3.0,0.0,5871.0,3043.0,757.0,1193.0,32.0,846.0


---

In [39]:
import pandas as pd

# Load CSV data
df_wea = pd.read_csv('weather-data.csv')
df_wea

,time,temperature_2m (°C),snowfall (cm),snow_depth (m),weather_code (wmo code),cloud_cover (%),wind_speed_10m (km/h),wind_direction_10m (°)
0,2014-01-01T00:00,4.5,0.0,0.0,3,100,6.1,298
1,2014-01-01T01:00,3.7,0.0,0.0,3,100,6.1,310
2,2014-01-01T02:00,3.2,0.0,0.0,3,100,6.0,327
3,2014-01-01T03:00,2.7,0.0,0.0,3,100,5.5,328
4,2014-01-01T04:00,2.0,0.0,0.0,3,100,6.1,332
...,...,...,...,...,...,...,...,...
96427,2024-12-31T19:00,18.7,0.0,NaN,0,17,23.3,243
96428,2024-12-31T20:00,18.7,0.0,NaN,0,2,26.6,257
96429,2024-12-31T21:00,17.7,0.0,NaN,0,0,25.3,265
96430,2024-12-31T22:00,15.9,0.0,NaN,0,0,22.2,275


In [40]:
import pandas as pd


# Load CSV data
df_wea = pd.read_csv('weather-data.csv')
df_wea['time'] = pd.to_datetime(df_wea['time'])  # or df_wea['timestamp']
# Extract date and time components from the timestamp
df_wea['date'] = df_wea['time'].dt.date
df_wea['hour'] = df_wea['time'].dt.hour
df_wea.rename(columns={'month': 'date'}, inplace=True)

# Group by date and calculate daily statistic
# Extract the month and year from the 'time' column to group by
df_wea['year'] = df_wea['time'].dt.year
df_wea['month'] = df_wea['time'].dt.month

df_wea.rename(columns={'temperature_2m (°C)': 'temperature_2m'}, inplace=True)
df_wea.rename(columns={'snowfall (cm)': 'snowfall'}, inplace=True)
df_wea.rename(columns={'snow_depth (m)': 'snow_depth'}, inplace=True)
df_wea.rename(columns={'cloud_cover (%)': 'cloud_cover'}, inplace=True)
df_wea.rename(columns={'wind_speed_10m (km/h)': 'wind_speed_10m'}, inplace=True)
df_wea.rename(columns={'wind_direction_10m (°)': 'wind_direction_10m'}, inplace=True)



df_wea = df_wea.groupby(['year', 'month']).agg({
    'temperature_2m': 'mean',
    'snowfall': 'sum',
    'snow_depth': 'mean',
    'cloud_cover': 'mean',
    'wind_speed_10m': 'mean',
    'wind_direction_10m': 'mean'
}).reset_index()

df_wea


,year,month,temperature_2m,snowfall,snow_depth,cloud_cover,wind_speed_10m,wind_direction_10m
0,2014,1,1.921505,4.41,0.004126,43.788978,13.489919,232.897849
1,2014,2,7.051786,11.55,0.006235,65.367560,12.793601,236.766369
2,2014,3,9.742876,0.00,0.000000,60.543011,12.783199,208.944892
3,2014,4,15.451111,0.00,0.000000,58.587500,13.065000,196.804167
4,2014,5,20.724597,0.00,0.000000,53.099462,9.122043,206.063172
...,...,...,...,...,...,...,...,...
127,2024,8,25.959005,0.00,0.000000,25.107527,10.002151,197.163978
128,2024,9,23.060000,0.00,0.000000,51.530556,13.089583,152.679167
129,2024,10,17.751344,0.00,0.000000,14.504032,10.080108,166.244624
130,2024,11,14.107083,0.00,0.000000,51.059722,11.963194,192.959722


In [41]:
df_wea.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 132 entries, 0 to 131
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   year                132 non-null    int32  
 1   month               132 non-null    int32  
 2   temperature_2m      132 non-null    float64
 3   snowfall            132 non-null    float64
 4   snow_depth          132 non-null    float64
 5   cloud_cover         132 non-null    float64
 6   wind_speed_10m      132 non-null    float64
 7   wind_direction_10m  132 non-null    float64
dtypes: float64(6), int32(2)
memory usage: 7.3 KB


---

In [42]:
fs = project.get_feature_store()

In [44]:
merged_data = pd.merge(
    df,
    df_wea,
    left_on=['year', 'month'],
    right_on=['year', 'month'],
    how='left'  # 'left' keeps all flight data even if weather is missing
)

In [45]:
merged_data

,year,month,carrier,carrier_name,arr_flights,arr_del15,carrier_ct,weather_ct,nas_ct,security_ct,...,weather_delay,nas_delay,security_delay,late_aircraft_delay,temperature_2m,snowfall,snow_depth,cloud_cover,wind_speed_10m,wind_direction_10m
0,2024,9,9E,Endeavor Air Inc.,2204.0,280.0,61.70,17.36,88.36,0.00,...,2297.0,3690.0,0.0,10556.0,23.060000,0.0,0.0,51.530556,13.089583,152.679167
1,2024,9,AA,American Airlines Network,709.0,159.0,63.21,3.02,35.25,0.07,...,238.0,1327.0,6.0,4521.0,23.060000,0.0,0.0,51.530556,13.089583,152.679167
2,2024,9,AS,Alaska Airlines Network,138.0,37.0,10.67,1.63,18.47,0.00,...,135.0,907.0,0.0,211.0,23.060000,0.0,0.0,51.530556,13.089583,152.679167
3,2024,9,B6,JetBlue Airways,276.0,57.0,26.27,0.98,18.84,0.00,...,40.0,657.0,0.0,910.0,23.060000,0.0,0.0,51.530556,13.089583,152.679167
4,2024,9,DL,Delta Air Lines Network,19151.0,2429.0,709.98,122.72,690.90,0.68,...,12036.0,26112.0,32.0,56716.0,23.060000,0.0,0.0,51.530556,13.089583,152.679167
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1599,2014,7,MQ,Envoy Air,81.0,21.0,3.28,0.20,11.39,0.00,...,8.0,535.0,0.0,222.0,24.931586,0.0,0.0,59.395161,8.178763,204.651882
1600,2014,7,OO,SkyWest Airlines Inc.,198.0,48.0,16.68,1.67,18.52,0.00,...,230.0,604.0,0.0,518.0,24.931586,0.0,0.0,59.395161,8.178763,204.651882
1601,2014,7,UA,United Air Lines Inc.,61.0,37.0,15.79,0.28,11.51,0.00,...,13.0,386.0,0.0,614.0,24.931586,0.0,0.0,59.395161,8.178763,204.651882
1602,2014,7,US,US Airways Inc.,474.0,111.0,55.69,7.03,28.91,1.20,...,757.0,1193.0,32.0,846.0,24.931586,0.0,0.0,59.395161,8.178763,204.651882


In [55]:
import numpy as np

#Create 'event_time' in DATE format for feature group
merged_data['event_time'] = pd.to_datetime(
    merged_data['year'].astype(str) + '-' + merged_data['month'].astype(str) + '-01'
).dt.date

#Cyclical encoding for month to capture cyclical patterns
merged_data['month_sin'] = np.sin(2 * np.pi * merged_data['month'] / 12)
merged_data['month_cos'] = np.cos(2 * np.pi * merged_data['month'] / 12)

#Create feature group
delay_data = fs.get_or_create_feature_group(
    name='airline_delay2',
    description='Flight delay data merged with weather conditions',
    version=1,
    primary_key=['year', 'month', 'carrier'],  # Ensure primary key uniqueness
    event_time="event_time"
)

In [56]:
delay_data.insert(merged_data, write_options={"wait_for_job": True})


Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1171606/fs/1162309/fg/1393628


Uploading Dataframe: 100.00% |██████████| Rows 1604/1604 | Elapsed Time: 00:02 | Remaining Time: 00:00


Launching job: airline_delay2_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1171606/jobs/named/airline_delay2_1_offline_fg_materialization/executions


(Job('airline_delay2_1_offline_fg_materialization', 'SPARK'), None)